In [37]:
import importlib
import numpy as np
import src.data_loaders.ADNI_A as ADNI_A
import src.data_processing.filterps as filterps
import src.data_processing.zfilterts as zfilterts
import LinHopfFit
from typing import Union
importlib.reload(LinHopfFit)

<module 'LinHopfFit' from '/Users/mellebolding/MSc/Thesis/linhopf_fdt_adni/LinHopfFit.py'>

In [ ]:
NPARCELLS = 40 # max 379
a_param = -0.02
gconst = 1.0
avec = a_param * np.ones(NPARCELLS)
sigma_mean = 0.45
sigma_ini = sigma_mean * np.ones(NPARCELLS)

## hyperparams
hopfParamsAdam = {
'verbose': True,
'fit_sigma': True,
'fit_a': True,
'competitive_coupling': False,
'allow_negative': False,
'loss_weights': {'mse_fc': 0.25, 'mse_covtau': 0.25, 'corr_fc': 0.25, 'corr_covtau': 0.25},
'lr_Ceff': 1e-4,
'lr_sigma': 1e-4,
'lr_a': 5e-4,
'beta1': 0.9,
'beta2': 0.999,
'epsilon': 1e-8,
'max_iter': 10,
'iter_check': 50,
'patience': 5,
'maxC': 0.2,
'TR': 3,
'g': 1,
'tau': 1.0,
}

In [ ]:
# Loading the timeseries data for all subjects and dividing them into groups

DL = ADNI_A.ADNI_A(normalizeBurden=False)
HC_IDs = DL.get_groupSubjects('HC')
HC_MRI = {}
HC_SC = {}
f_diff_HC = {}
for subject in HC_IDs:
    data = DL.get_subjectData(subject,printInfo=False)
    HC_MRI[subject] = data[subject]['timeseries'].T
    HC_SC[subject] = data[subject]['SC']
    f_diff_HC[subject] = filterps.calc_H_freq(HC_MRI[subject], 3000, filterps.FiltPowSpetraVersion.v2021)[0]
MCI_IDs = DL.get_groupSubjects('MCI')
MCI_MRI = {}
MCI_SC = {}
f_diff_MCI = {}
for subject in MCI_IDs:
    data = DL.get_subjectData(subject,printInfo=False)
    MCI_MRI[subject] = data[subject]['timeseries'].T
    MCI_SC[subject] = data[subject]['SC']
    f_diff_MCI[subject] = filterps.calc_H_freq(MCI_MRI[subject], 3000, filterps.FiltPowSpetraVersion.v2021)[0]
AD_IDs = DL.get_groupSubjects('AD')
AD_MRI = {}
AD_SC = {}
f_diff_AD = {}
for subject in AD_IDs:
    data = DL.get_subjectData(subject,printInfo=False)
    AD_MRI[subject] = data[subject]['timeseries'].T
    AD_SC[subject] = data[subject]['SC']
    f_diff_AD[subject] = filterps.calc_H_freq(AD_MRI[subject], 3000, filterps.FiltPowSpetraVersion.v2021)[0]

all_MRI = {**HC_MRI, **MCI_MRI, **AD_MRI}
all_SC = {**HC_SC, **MCI_SC, **AD_SC}
all_ID= [*HC_IDs, *MCI_IDs, *AD_IDs]
all_f_diff = {**f_diff_HC, **f_diff_MCI, **f_diff_AD}


In [12]:
min_ntimes = min(all_MRI[subj_id].shape[0] for subj_id in all_ID)
ts_gr_arr = np.zeros((len(all_ID), NPARCELLS, min_ntimes))

for sub in range(len(all_ID)):
    subj_id = all_ID[sub]
    ts_gr_arr[sub,:,:] = all_MRI[subj_id][:min_ntimes,:NPARCELLS].T.copy() 
TSemp_zsc = zfilterts.zscore_time_series(ts_gr_arr, mode='global', detrend=True)[:,:NPARCELLS,:].copy() #mode: parcel, global, none

In [23]:
param_grid = {
    'lr_Ceff': [1,1e-5, 5e-5, 1e-4, 5e-4],
    'lr_sigma': [1e-5, 5e-5, 1e-4, 5e-4],
    'lr_a': [1e-4, 5e-4, 1e-3, 5e-3],
    'beta1': [0.85, 0.9, 0.95],
    'beta2': [0.99, 0.999],
    'epsilon': [1e-9, 1e-8, 1e-7],
}

def sensitivity_analysis():
    """Test each parameter individually to see which matter most"""
    baseline = {
        'lr_Ceff': 1e-4,
        'lr_sigma': 1e-4,
        'lr_a': 5e-4,
        'beta1': 0.9,
        'beta2': 0.999,
        'epsilon': 1e-8,
        'max_iter': 1000,
        'iter_check': 50,
        'patience': 5,
        'maxC': 0.2,
        'TR': 3,
        'g': 1,
        'tau': 1.0,
        'verbose': False,
        'fit_sigma': True,
        'fit_a': True,
        'competitive_coupling': False,
        'allow_negative': False,
        'loss_weights': {'mse_fc': 0.25, 'mse_covtau': 0.25, 'corr_fc': 0.25, 'corr_covtau': 0.25},
    }
    
    sensitivity = {}
    
    for param_name, param_values in param_grid.items():
        param_results = []
        
        for val in param_values:
            params = baseline.copy()
            params[param_name] = val
            
            mse_list = []
            for sub in range(len(all_ID)):
                subj_id = all_ID[sub]
                SC = all_SC[subj_id][:NPARCELLS, :NPARCELLS]
                f_diff = all_f_diff[subj_id][:NPARCELLS]
                
                model = LinHopfFit.LinearHopfModel(C=SC, f_diff=f_diff, 
                                                   sigma=sigma_ini, a=avec, 
                                                   **params)
                model.setup_optimization(optimizer_method='adam', **params)
                Ceff_sub, sigma_sub, a_sub, err_mse= model.fit(TSemp_zsc[sub])
                mse_list.append(err_mse)
            
            avg_mse = np.mean(mse_list)
            param_results.append((val, avg_mse))
            print(f"{param_name} = {val}: MSE = {avg_mse:.6f}")
        
        sensitivity[param_name] = param_results
    
    return sensitivity

sensitivity = sensitivity_analysis()

lr_Ceff = 1: MSE = 0.124189
lr_Ceff = 1e-05: MSE = 0.156948
lr_Ceff = 5e-05: MSE = 0.137734
lr_Ceff = 0.0001: MSE = 0.130431
lr_Ceff = 0.0005: MSE = 0.119666
lr_sigma = 1e-05: MSE = 0.128834
lr_sigma = 5e-05: MSE = 0.129553
lr_sigma = 0.0001: MSE = 0.130431
lr_sigma = 0.0005: MSE = 0.140807
lr_a = 0.0001: MSE = 0.124471
lr_a = 0.0005: MSE = 0.130431
lr_a = 0.001: MSE = 0.136566
lr_a = 0.005: MSE = 0.146320
beta1 = 0.85: MSE = 0.130554
beta1 = 0.9: MSE = 0.130431
beta1 = 0.95: MSE = 0.130006
beta2 = 0.99: MSE = 0.132565
beta2 = 0.999: MSE = 0.130431
epsilon = 1e-09: MSE = 0.130431
epsilon = 1e-08: MSE = 0.130431
epsilon = 1e-07: MSE = 0.130431


In [ ]:
def bayesian_optimization(n_trials=100):
    """Use Optuna for Bayesian optimization"""
    import optuna
    
    def objective(trial):
        params = {
            'lr_Ceff': trial.suggest_float('lr_Ceff', 1e-5, 1e-3, log=True),
            'lr_sigma': trial.suggest_float('lr_sigma', 1e-5, 1e-3, log=True),
            'lr_a': trial.suggest_float('lr_a', 1e-4, 5e-3, log=True),
            'beta1': trial.suggest_float('beta1', 0.85, 0.95),
            'beta2': trial.suggest_float('beta2', 0.99, 0.999),
            'epsilon': trial.suggest_float('epsilon', 1e-9, 1e-7, log=True),
            'max_iter': 10000,
            'iter_check': 50,
            'patience': 5,
            'maxC': 0.2,
            'TR': 3,
            'g': 1,
            'tau': 1.0,
            'verbose': False,
            'fit_sigma': True,
            'fit_a': True,
            'competitive_coupling': False,
            'allow_negative': False,
            'loss_weights': {'mse_fc': 0.25, 'mse_covtau': 0.25, 'corr_fc': 0.25, 'corr_covtau': 0.25},
        }
        
        mse_list = []
        for sub in range(len(all_ID)):
            subj_id = all_ID[sub]
            SC = all_SC[subj_id][:NPARCELLS, :NPARCELLS]
            f_diff = all_f_diff[subj_id][:NPARCELLS]
            
            model = LinHopfFit.LinearHopfModel(C=SC, f_diff=f_diff, 
                                               sigma=sigma_ini, a=avec, 
                                               **params)
            model.setup_optimization(optimizer_method='adam',**params)
            Ceff_sub, sigma_sub, a_sub, err_mse = model.fit(TSemp_zsc[sub])
            mse_list.append(err_mse)
        
        return np.mean(mse_list)
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    
    print(f"\nBest MSE: {study.best_value:.6f}")
    print("Best parameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")
    
    return study.best_params, study

bayesian_optimization(n_trials=5)

[I 2025-10-02 15:31:36,004] A new study created in memory with name: no-name-8ff26822-1538-4fda-969e-ad98d2e64265


  0%|          | 0/50 [00:00<?, ?it/s]

Early stopping: no improvement for 5 checks
Error=0.026830, CorrFC=0.819, CorrCOV=0.738
Early stopping: no improvement for 5 checks
Error=0.023512, CorrFC=0.844, CorrCOV=0.675
Early stopping: no improvement for 5 checks
Error=0.030786, CorrFC=0.849, CorrCOV=0.741
Early stopping: no improvement for 5 checks
Error=0.030319, CorrFC=0.810, CorrCOV=0.734
Early stopping: no improvement for 5 checks
Error=0.032354, CorrFC=0.876, CorrCOV=0.757
Early stopping: no improvement for 5 checks
Error=0.034551, CorrFC=0.877, CorrCOV=0.542
Early stopping: no improvement for 5 checks
Error=0.032048, CorrFC=0.870, CorrCOV=0.849
Early stopping: no improvement for 5 checks
Error=0.053938, CorrFC=0.888, CorrCOV=0.764
Early stopping: no improvement for 5 checks
Error=0.038747, CorrFC=0.839, CorrCOV=0.708
Early stopping: no improvement for 5 checks
Error=0.025564, CorrFC=0.866, CorrCOV=0.630
Early stopping: no improvement for 5 checks
Error=0.024870, CorrFC=0.867, CorrCOV=0.639
Early stopping: no improvement f

KeyboardInterrupt: 

In [40]:
# here we only run the subjects, group would take average SC and the group-specific TSemp_zscs (model.fit can take multiple timeseries at once)


for sub in range(len(all_ID)):
    subj_id = all_ID[sub]
    SC = all_SC[subj_id][:NPARCELLS, :NPARCELLS]
    f_diff = all_f_diff[subj_id][:NPARCELLS]
    #SC_N = (SC / np.max(SC)) * 0.2
    model = LinHopfFit.LinearHopfModel(C=SC, f_diff=f_diff, sigma=sigma_ini, a=avec, **hopfParamsAdam)
    model.setup_optimization(optimizer_method='lbfgs',**hopfParamsAdam)
    Ceff_sub, sigma_sub, a_sub, err_mse, corr_fc, corr_cov = model.fit(TSemp_zsc[sub])




KeyboardInterrupt: 